In [2]:
from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from scipy.stats import zscore
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from keras.callbacks import ModelCheckpoint
import tensorflow as tf
from sklearn.model_selection import KFold
from tensorflow.keras.layers import Dense, Activation
from keras.utils import to_categorical
from sklearn.model_selection import StratifiedKFold

Using TensorFlow backend.


In [3]:
# LER OS DADOS E SEPARA-LOS
dataset = pd.read_csv('dadosCORRETOS.csv', header=1)

In [4]:
x = dataset.iloc[:,3:54676].values
y = dataset.iloc[:,2].values
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
# Cross-Validate
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=7)
oos_y = []
oos_pred = []
models = []
cvscores = []
history = []
fold = 0
for train, test in kf.split(X_train, y_train):
    fold+=1
    print(f"Fold #{fold}")
    filepath= 'teste2/'+str(fold)+"-Correto4.hdf5"
    filepath2= 'teste2/'+str(fold)+"-Correto5.hdf5"
    filepath3= 'teste2/'+str(fold)+"-Correto6.hdf5"
    filepath4= 'teste2/'+str(fold)+"-Correto7.hdf5"
    mcp_save = ModelCheckpoint(filepath ,save_best_only=True, verbose=0, monitor='val_loss', mode='min')
    acura_save = ModelCheckpoint(filepath2 ,save_best_only=True, verbose=0, monitor='val_accuracy', mode='max')
    mcp_savew = ModelCheckpoint(filepath3 ,save_best_only=True, verbose=0, monitor='loss', mode='min')
    acura_savew = ModelCheckpoint(filepath4 ,save_best_only=True, verbose=0, monitor='accuracy', mode='max')
    
    model = tf.keras.Sequential()
    model.add(Dense(64, input_dim=x.shape[1], activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])
    #   
    history.append(model.fit(X_train[train], y_train[train], validation_data=(X_train[test], y_train[test]), 
        # salvar parametros melhor acuracy # callbacks= [acura_save],
                             verbose=2, batch_size=200, epochs=100))
    
    pred = model.predict(X_test)
    
    oos_y.append(y_test)
    oos_pred.append(pred)
    models.append(model)
    score = np.sqrt(metrics.mean_squared_error(pred,y_test))
    print(f"Fold score (RMSE): {score}")
    
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
score = np.sqrt(metrics.mean_squared_error(oos_pred,oos_y))
print(f"Final, out of sample score (RMSE): {score}")

Fold #1
Train on 102 samples, validate on 27 samples
Epoch 1/2000
102/102 - 2s - loss: 166.4199 - accuracy: 0.4314 - val_loss: 15151.5205 - val_accuracy: 0.5556
Epoch 2/2000
102/102 - 0s - loss: 16255.3066 - accuracy: 0.5490 - val_loss: 7695.4761 - val_accuracy: 0.5556
Epoch 3/2000
102/102 - 0s - loss: 8263.9697 - accuracy: 0.5490 - val_loss: 207.1823 - val_accuracy: 0.4815
Epoch 4/2000
102/102 - 0s - loss: 206.3633 - accuracy: 0.4510 - val_loss: 195.1424 - val_accuracy: 0.5185
Epoch 5/2000
102/102 - 0s - loss: 222.8218 - accuracy: 0.5294 - val_loss: 1631.1554 - val_accuracy: 0.4444
Epoch 6/2000
102/102 - 0s - loss: 1636.0278 - accuracy: 0.4510 - val_loss: 123.1901 - val_accuracy: 0.5926
Epoch 7/2000
102/102 - 0s - loss: 127.4946 - accuracy: 0.5588 - val_loss: 224.9359 - val_accuracy: 0.5556
Epoch 8/2000
102/102 - 0s - loss: 234.5567 - accuracy: 0.5490 - val_loss: 1040.3776 - val_accuracy: 0.4444
Epoch 9/2000
102/102 - 0s - loss: 1019.7173 - accuracy: 0.4510 - val_loss: 221.1190 - val_

Epoch 77/2000
102/102 - 0s - loss: 377.2521 - accuracy: 0.4706 - val_loss: 111.1451 - val_accuracy: 0.5185
Epoch 78/2000
102/102 - 0s - loss: 69.6996 - accuracy: 0.6863 - val_loss: 317.7608 - val_accuracy: 0.5926
Epoch 79/2000
102/102 - 0s - loss: 323.0217 - accuracy: 0.5588 - val_loss: 279.9481 - val_accuracy: 0.5926
Epoch 80/2000
102/102 - 0s - loss: 281.7424 - accuracy: 0.5784 - val_loss: 108.0667 - val_accuracy: 0.5556
Epoch 81/2000
102/102 - 0s - loss: 56.4873 - accuracy: 0.6961 - val_loss: 407.0777 - val_accuracy: 0.4444
Epoch 82/2000
102/102 - 0s - loss: 283.6877 - accuracy: 0.4804 - val_loss: 234.5585 - val_accuracy: 0.5185
Epoch 83/2000
102/102 - 0s - loss: 133.0632 - accuracy: 0.5882 - val_loss: 198.3610 - val_accuracy: 0.5926
Epoch 84/2000
102/102 - 0s - loss: 184.2590 - accuracy: 0.5882 - val_loss: 262.8804 - val_accuracy: 0.5926
Epoch 85/2000
102/102 - 0s - loss: 260.1616 - accuracy: 0.5686 - val_loss: 113.1965 - val_accuracy: 0.5185
Epoch 86/2000
102/102 - 0s - loss: 59.3

Epoch 155/2000
102/102 - 0s - loss: 1.1179 - accuracy: 0.9020 - val_loss: 63.6111 - val_accuracy: 0.4815
Epoch 156/2000
102/102 - 0s - loss: 1.0980 - accuracy: 0.8922 - val_loss: 63.2523 - val_accuracy: 0.4815
Epoch 157/2000
102/102 - 0s - loss: 1.0720 - accuracy: 0.8922 - val_loss: 64.1159 - val_accuracy: 0.4815
Epoch 158/2000
102/102 - 0s - loss: 1.0422 - accuracy: 0.8922 - val_loss: 65.1709 - val_accuracy: 0.4815
Epoch 159/2000
102/102 - 0s - loss: 1.0167 - accuracy: 0.8922 - val_loss: 65.8800 - val_accuracy: 0.4815
Epoch 160/2000
102/102 - 0s - loss: 0.9933 - accuracy: 0.8922 - val_loss: 65.7648 - val_accuracy: 0.4815
Epoch 161/2000
102/102 - 0s - loss: 0.9707 - accuracy: 0.8824 - val_loss: 67.4026 - val_accuracy: 0.4815
Epoch 162/2000
102/102 - 0s - loss: 0.9844 - accuracy: 0.9020 - val_loss: 66.3714 - val_accuracy: 0.4815
Epoch 163/2000
102/102 - 0s - loss: 0.9277 - accuracy: 0.8922 - val_loss: 68.1445 - val_accuracy: 0.5185
Epoch 164/2000
102/102 - 0s - loss: 0.9422 - accuracy: 

Epoch 233/2000
102/102 - 0s - loss: 169.2482 - accuracy: 0.5784 - val_loss: 58.4791 - val_accuracy: 0.5185
Epoch 234/2000
102/102 - 0s - loss: 12.2472 - accuracy: 0.7941 - val_loss: 363.6196 - val_accuracy: 0.4444
Epoch 235/2000
102/102 - 0s - loss: 244.2516 - accuracy: 0.4706 - val_loss: 75.0809 - val_accuracy: 0.5185
Epoch 236/2000
102/102 - 0s - loss: 4.3588 - accuracy: 0.8627 - val_loss: 236.6535 - val_accuracy: 0.5926
Epoch 237/2000
102/102 - 0s - loss: 198.3843 - accuracy: 0.5686 - val_loss: 119.7812 - val_accuracy: 0.6296
Epoch 238/2000
102/102 - 0s - loss: 68.6986 - accuracy: 0.6373 - val_loss: 426.8275 - val_accuracy: 0.4444
Epoch 239/2000
102/102 - 0s - loss: 301.6809 - accuracy: 0.4510 - val_loss: 204.0982 - val_accuracy: 0.4815
Epoch 240/2000
102/102 - 0s - loss: 87.9728 - accuracy: 0.5588 - val_loss: 356.0195 - val_accuracy: 0.5556
Epoch 241/2000
102/102 - 0s - loss: 318.1178 - accuracy: 0.5588 - val_loss: 443.1404 - val_accuracy: 0.5556
Epoch 242/2000
102/102 - 0s - loss:

Epoch 310/2000
102/102 - 0s - loss: 1.1578 - accuracy: 0.9314 - val_loss: 83.4965 - val_accuracy: 0.4815
Epoch 311/2000
102/102 - 0s - loss: 0.4296 - accuracy: 0.9118 - val_loss: 75.1398 - val_accuracy: 0.5556
Epoch 312/2000
102/102 - 0s - loss: 2.4597 - accuracy: 0.8922 - val_loss: 75.2711 - val_accuracy: 0.4815
Epoch 313/2000
102/102 - 0s - loss: 2.0130 - accuracy: 0.8725 - val_loss: 83.8620 - val_accuracy: 0.4815
Epoch 314/2000
102/102 - 0s - loss: 0.5218 - accuracy: 0.9314 - val_loss: 90.9204 - val_accuracy: 0.5185
Epoch 315/2000
102/102 - 0s - loss: 1.0769 - accuracy: 0.9314 - val_loss: 94.1927 - val_accuracy: 0.5185
Epoch 316/2000
102/102 - 0s - loss: 1.3339 - accuracy: 0.9314 - val_loss: 93.9974 - val_accuracy: 0.5185
Epoch 317/2000
102/102 - 0s - loss: 1.3147 - accuracy: 0.9314 - val_loss: 90.6824 - val_accuracy: 0.5185
Epoch 318/2000
102/102 - 0s - loss: 1.0493 - accuracy: 0.9314 - val_loss: 84.6327 - val_accuracy: 0.5185
Epoch 319/2000
102/102 - 0s - loss: 0.5845 - accuracy: 

...

Epoch 1347/2000
103/103 - 0s - loss: 0.0437 - accuracy: 1.0000 - val_loss: 185.2657 - val_accuracy: 0.5000
Epoch 1348/2000
103/103 - 0s - loss: 0.0436 - accuracy: 1.0000 - val_loss: 185.2548 - val_accuracy: 0.5000
Epoch 1349/2000
103/103 - 0s - loss: 0.0436 - accuracy: 1.0000 - val_loss: 185.2437 - val_accuracy: 0.5000
Epoch 1350/2000
103/103 - 0s - loss: 0.0435 - accuracy: 1.0000 - val_loss: 185.2324 - val_accuracy: 0.5000
Epoch 1351/2000
103/103 - 0s - loss: 0.0435 - accuracy: 1.0000 - val_loss: 185.2219 - val_accuracy: 0.5000
Epoch 1352/2000
103/103 - 0s - loss: 0.0434 - accuracy: 1.0000 - val_loss: 185.2111 - val_accuracy: 0.5000
Epoch 1353/2000
103/103 - 0s - loss: 0.0434 - accuracy: 1.0000 - val_loss: 185.1997 - val_accuracy: 0.5000
Epoch 1354/2000
103/103 - 0s - loss: 0.0433 - accuracy: 1.0000 - val_loss: 185.1889 - val_accuracy: 0.5000
Epoch 1355/2000
103/103 - 0s - loss: 0.0433 - accuracy: 1.0000 - val_loss: 185.1781 - val_accuracy: 0.5000
Epoch 1356/2000
103/103 - 0s - loss: 

Epoch 1424/2000
103/103 - 0s - loss: 0.0401 - accuracy: 1.0000 - val_loss: 184.4354 - val_accuracy: 0.5000
Epoch 1425/2000
103/103 - 0s - loss: 0.0401 - accuracy: 1.0000 - val_loss: 184.4253 - val_accuracy: 0.5000
Epoch 1426/2000
103/103 - 0s - loss: 0.0400 - accuracy: 1.0000 - val_loss: 184.4146 - val_accuracy: 0.5000
Epoch 1427/2000
103/103 - 0s - loss: 0.0400 - accuracy: 1.0000 - val_loss: 184.4038 - val_accuracy: 0.5000
Epoch 1428/2000
103/103 - 0s - loss: 0.0400 - accuracy: 1.0000 - val_loss: 184.3930 - val_accuracy: 0.5000
Epoch 1429/2000
103/103 - 0s - loss: 0.0399 - accuracy: 1.0000 - val_loss: 184.3821 - val_accuracy: 0.5000
Epoch 1430/2000
103/103 - 0s - loss: 0.0399 - accuracy: 1.0000 - val_loss: 184.3721 - val_accuracy: 0.5000
Epoch 1431/2000
103/103 - 0s - loss: 0.0398 - accuracy: 1.0000 - val_loss: 184.3613 - val_accuracy: 0.5000
Epoch 1432/2000
103/103 - 0s - loss: 0.0398 - accuracy: 1.0000 - val_loss: 184.3509 - val_accuracy: 0.5000
Epoch 1433/2000
103/103 - 0s - loss: 

Epoch 1501/2000
103/103 - 0s - loss: 0.0370 - accuracy: 1.0000 - val_loss: 183.6397 - val_accuracy: 0.5000
Epoch 1502/2000
103/103 - 0s - loss: 0.0370 - accuracy: 1.0000 - val_loss: 183.6293 - val_accuracy: 0.5000
Epoch 1503/2000
103/103 - 0s - loss: 0.0369 - accuracy: 1.0000 - val_loss: 183.6195 - val_accuracy: 0.5000
Epoch 1504/2000
103/103 - 0s - loss: 0.0369 - accuracy: 1.0000 - val_loss: 183.6097 - val_accuracy: 0.5000
Epoch 1505/2000
103/103 - 0s - loss: 0.0369 - accuracy: 1.0000 - val_loss: 183.5999 - val_accuracy: 0.5000
Epoch 1506/2000
103/103 - 0s - loss: 0.0368 - accuracy: 1.0000 - val_loss: 183.5892 - val_accuracy: 0.5000
Epoch 1507/2000
103/103 - 0s - loss: 0.0368 - accuracy: 1.0000 - val_loss: 183.5786 - val_accuracy: 0.5000
Epoch 1508/2000
103/103 - 0s - loss: 0.0367 - accuracy: 1.0000 - val_loss: 183.5691 - val_accuracy: 0.5000
Epoch 1509/2000
103/103 - 0s - loss: 0.0367 - accuracy: 1.0000 - val_loss: 183.5589 - val_accuracy: 0.5000
Epoch 1510/2000
103/103 - 0s - loss: 

Epoch 1578/2000
103/103 - 0s - loss: 0.0342 - accuracy: 1.0000 - val_loss: 182.8781 - val_accuracy: 0.5000
Epoch 1579/2000
103/103 - 0s - loss: 0.0342 - accuracy: 1.0000 - val_loss: 182.8689 - val_accuracy: 0.5000
Epoch 1580/2000
103/103 - 0s - loss: 0.0342 - accuracy: 1.0000 - val_loss: 182.8590 - val_accuracy: 0.5000
Epoch 1581/2000
103/103 - 0s - loss: 0.0341 - accuracy: 1.0000 - val_loss: 182.8494 - val_accuracy: 0.5000
Epoch 1582/2000
103/103 - 0s - loss: 0.0341 - accuracy: 1.0000 - val_loss: 182.8399 - val_accuracy: 0.5000
Epoch 1583/2000
103/103 - 0s - loss: 0.0340 - accuracy: 1.0000 - val_loss: 182.8304 - val_accuracy: 0.5000
Epoch 1584/2000
103/103 - 0s - loss: 0.0340 - accuracy: 1.0000 - val_loss: 182.8208 - val_accuracy: 0.5000
Epoch 1585/2000
103/103 - 0s - loss: 0.0340 - accuracy: 1.0000 - val_loss: 182.8113 - val_accuracy: 0.5000
Epoch 1586/2000
103/103 - 0s - loss: 0.0339 - accuracy: 1.0000 - val_loss: 182.8012 - val_accuracy: 0.5000
Epoch 1587/2000
103/103 - 0s - loss: 

Epoch 1655/2000
103/103 - 0s - loss: 0.0317 - accuracy: 1.0000 - val_loss: 182.1484 - val_accuracy: 0.5000
Epoch 1656/2000
103/103 - 0s - loss: 0.0317 - accuracy: 1.0000 - val_loss: 182.1391 - val_accuracy: 0.5000
Epoch 1657/2000
103/103 - 0s - loss: 0.0317 - accuracy: 1.0000 - val_loss: 182.1296 - val_accuracy: 0.5000
Epoch 1658/2000
103/103 - 0s - loss: 0.0316 - accuracy: 1.0000 - val_loss: 182.1202 - val_accuracy: 0.5000
Epoch 1659/2000
103/103 - 0s - loss: 0.0316 - accuracy: 1.0000 - val_loss: 182.1106 - val_accuracy: 0.5000
Epoch 1660/2000
103/103 - 0s - loss: 0.0316 - accuracy: 1.0000 - val_loss: 182.1024 - val_accuracy: 0.5000
Epoch 1661/2000
103/103 - 0s - loss: 0.0315 - accuracy: 1.0000 - val_loss: 182.0930 - val_accuracy: 0.5000
Epoch 1662/2000
103/103 - 0s - loss: 0.0315 - accuracy: 1.0000 - val_loss: 182.0842 - val_accuracy: 0.5000
Epoch 1663/2000
103/103 - 0s - loss: 0.0315 - accuracy: 1.0000 - val_loss: 182.0752 - val_accuracy: 0.5000
Epoch 1664/2000
103/103 - 0s - loss: 

In [ ]:
# Mostrar melhor resultado obtidos em cada fold

for result in history:
    validation_acc = np.amax(result.history['val_accuracy'])
    print(validation_acc)

In [ ]:
# Mostrar acuracia de cada fold nos dados de teste

for model in models:
    _, accuracy = model.evaluate(X_test, y_test, verbose = 0)
    print('Accuracy: %.2f' % (accuracy*100))

In [ ]:
def plot_history(history):
    loss_list = [s for s in history.history.keys() if 'loss' in s and 'val' not in s]
    val_loss_list = [s for s in history.history.keys() if 'loss' in s and 'val' in s]
    acc_list = [s for s in history.history.keys() if 'acc' in s and 'val' not in s]
    val_acc_list = [s for s in history.history.keys() if 'acc' in s and 'val' in s]
    
    if len(loss_list) == 0:
        print('Loss is missing in history')
        return 
    
    ## As loss always exists
    epochs = range(1,len(history.history[loss_list[0]]) + 1)
    
    ## Loss
    plt.figure(1)
    for l in loss_list:
        plt.plot(epochs, history.history[l], 'b', label='Training loss (' + str(str(format(history.history[l][-1],'.5f'))+')'))
    for l in val_loss_list:
        plt.plot(epochs, history.history[l], 'g', label='Validation loss (' + str(str(format(history.history[l][-1],'.5f'))+')'))
    
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    
    ## Accuracy
    plt.figure(2)
    for l in acc_list:
        plt.plot(epochs, history.history[l], 'b', label='Training accuracy (' + str(format(history.history[l][-1],'.5f'))+')')
    for l in val_acc_list:    
        plt.plot(epochs, history.history[l], 'g', label='Validation accuracy (' + str(format(history.history[l][-1],'.5f'))+')')

    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

In [ ]:
# Plotar grafico de cada fold

for historico in history:
    plot_history(historico)